# Evaluating Debiasing Methods Using Real Life Treatments

To further assess the real-world utility of our debiasing framework, we adapt an experimental setup inspired by Cindy’s causal inference framework, which provides treatments and known outcomes constructed from DHS survey data.

This notebook will prepare the data needed for the experiments. The full output will be saved in the `aid_data/interim` directory. 

**Note:** Due to GitHub's file size limitations, the shapefile required for adding Admin2 regions is not included in this repository. You can download the complete dataset from [GADM](https://gadm.org/download_world.html)


In [1]:
import os
import pandas as pd
import re
import shutil
import numpy as np
import itertools
import configparser
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

tqdm.pandas()

sns.set_theme()

# Read config file
config = configparser.ConfigParser()
config.read('../config.ini')

DATA_DIR = config['PATHS']['DATA_DIR']
MAPBOX_TOKEN = config['ACCESS TOKENS']['MAPBOX']

aid_data_dir = os.path.join(DATA_DIR, 'aid_data')

# If not done already, move the Aid data (../aid_data) to the data directory
if not os.path.exists(aid_data_dir):
    shutil.copytree('../aid_data', aid_data_dir)

In [2]:
df = pd.read_csv(os.path.join(DATA_DIR, 'dhs_with_imgs.csv'))

dhs_codes = df['cluster_id'].apply(lambda x: x.split('.')[0])

with open(os.path.join(aid_data_dir, 'data/dhs_to_iso3.json'), 'r') as f:
    dhs_to_iso3s = json.load(f)

df['iso3'] = dhs_codes.apply(lambda x: dhs_to_iso3s[x])

## World Bank

We get information about the World Bank projects from https://www.aiddata.org/data/world-bank-geocoded-research-release-level-1-v1-4-2

In [3]:
wb_proj = pd.read_csv(os.path.join(aid_data_dir, 'data/AiddataWB1.4.2/projects.csv'))
wb_locs = pd.read_csv(os.path.join(aid_data_dir, 'data/AiddataWB1.4.2/locations.csv'))
wb_anc = pd.read_csv(os.path.join(aid_data_dir, 'data/AiddataWB1.4.2/projects_ancillary.csv'))

/local/tmp.4856116/ipykernel_125576/1647210345.py:3: DtypeWarning: Columns (87,88,98,126,133,154,158,163) have mixed types. Specify dtype option on import or set low_memory=False.
  wb_anc = pd.read_csv(os.path.join(aid_data_dir, 'data/AiddataWB1.4.2/projects_ancillary.csv'))


Merge the projects with location information

In [4]:
wb_proj = wb_proj.merge(wb_locs, on="project_id", how="inner")
wb_proj

,project_id,is_geocoded,project_title,start_actual_isodate,start_actual_type,end_actual_isodate,end_actual_type,donors,donors_iso3,recipients,...,geoname_id,place_name,latitude,longitude,location_type_code,location_type_name,gazetteer_adm_code,gazetteer_adm_name,location_class,geographic_exactness
0,P124054,1,ML-Strengthening Reproductive Health,2011-12-20,start-actual,2017-02-28,end-actual,World Bank,DAC,Mali,...,2453866,Mali,18.00000,-2.00000,PCLI,independent political entity,6295630|6255146|ML,Earth|Africa|Mali,1,1
1,P124054,1,ML-Strengthening Reproductive Health,2011-12-20,start-actual,2017-02-28,end-actual,World Bank,DAC,Mali,...,2460594,Bamako Region,12.65000,-8.00000,ADM1,first-order administrative division,6295630|6255146|ML|01,Earth|Africa|Mali|Bamako Region,1,1
2,P124054,1,ML-Strengthening Reproductive Health,2011-12-20,start-actual,2017-02-28,end-actual,World Bank,DAC,Mali,...,2454532,Koulikoro,13.80000,-7.50000,ADM1,first-order administrative division,6295630|6255146|ML|07,Earth|Africa|Mali|Koulikoro,1,1
3,P124054,1,ML-Strengthening Reproductive Health,2011-12-20,start-actual,2017-02-28,end-actual,World Bank,DAC,Mali,...,2451477,Ségou,13.60000,-5.90000,ADM1,first-order administrative division,6295630|6255146|ML|05,Earth|Africa|Mali|Ségou,1,1
4,P124054,1,ML-Strengthening Reproductive Health,2011-12-20,start-actual,2017-02-28,end-actual,World Bank,DAC,Mali,...,2451184,Sikasso,11.30000,-6.20000,ADM1,first-order administrative division,6295630|6255146|ML|06,Earth|Africa|Mali|Sikasso,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61238,P000755,1,ET ROAD SEC. DEV. PROG..,1998-01-15,start-actual,2005-05-31,end-actual,World Bank,DAC,Ethiopia,...,7670666,Metemma,13.00000,36.25000,ADM3,third-order administrative division,6295630|6255146|ET|46|7670666,Earth|Africa|Ethiopia|Amhara|Metemma,1,1
61239,P000755,1,ET ROAD SEC. DEV. PROG..,1998-01-15,start-actual,2005-05-31,end-actual,World Bank,DAC,Ethiopia,...,343508,Ārjo,8.81667,36.46667,MT,mountain,6295630|6255146|ET|51|343508,Earth|Africa|Ethiopia|Oromiya|Ārjo,4,1
61240,P000755,1,ET ROAD SEC. DEV. PROG..,1998-01-15,start-actual,2005-05-31,end-actual,World Bank,DAC,Ethiopia,...,342787,Bambesi,9.71667,34.66667,MT,mountain,6295630|6255146|ET|47|342787,Earth|Africa|Ethiopia|Bīnshangul Gumuz|Bambesi,4,1
61241,P000755,1,ET ROAD SEC. DEV. PROG..,1998-01-15,start-actual,2005-05-31,end-actual,World Bank,DAC,Ethiopia,...,332816,Konso,5.25000,37.50000,LCTY,locality,6295630|6255146|ET|54|332816,"Earth|Africa|Ethiopia|Southern Nations, Nation...",3,1


Only retain the WB projects located in Africa

In [5]:
african_proj_ids = wb_anc.loc[wb_anc["REGION"] == "AFRICA"]["PROJECT ID"]
wb_proj = wb_proj[wb_proj["project_id"].isin(african_proj_ids)].copy()

Filter by start year and precision code

In [6]:
wb_proj = wb_proj[
    (wb_proj["transactions_start_year"] >= 2002) &
    (wb_proj["transactions_start_year"] <= 2014) &
    (wb_proj["precision_code"].isin([1, 2, 3]))
].reset_index(drop=True)

In [7]:
wb_proj["site_iso2"] = (
    wb_proj["gazetteer_adm_code"]
    .str.extract(r"\|([A-Z]{2})\|", expand=False)
)

# One of the projects has a missing iso2 code, but it is in Benin
wb_proj.loc[7627, "site_iso2"] = "BJ"

In [8]:

with open(os.path.join(aid_data_dir, 'data/africa_isos.json'), 'r') as f:
    iso2_to_iso3s = json.load(f)

wb_proj["site_iso3"] = wb_proj["site_iso2"].apply(lambda x: iso2_to_iso3s[x])

In [9]:
# Get country names from the iso3 codes. Only include countries that are in the dhs dataset
iso3_to_dhs_country = df.groupby('iso3')['country'].first().to_dict()
wb_proj['country'] = wb_proj['site_iso3'].apply(lambda x: iso3_to_dhs_country[x] if x in iso3_to_dhs_country else None)

Classify each project using the OECD sector classifications. Use the highest available "parent code" for each aid sector, e.g. classify projects with both code 11120 (Education facilities and training) and code 11130 (Teacher training) as simply code 110 (Education).

In [10]:
oecd_sect_df = pd.read_csv(os.path.join(aid_data_dir, 'data/OECD/DevFi_Classification.csv'))
oecd_sect_df

,sect_code,sect_code_name,parent_code
0,110,Education,NaN
1,111,"Education, Level Unspecified",110.0
2,11110,Education policy and administrative management,111.0
3,11120,Education facilities and training,111.0
4,11130,Teacher training,111.0
...,...,...,...
359,93010,Refugees/asylum seekers in donor countries - a...,930.0
360,998,Unallocated / Unspecified,NaN
361,99810,Sectors not specified,998.0
362,99820,Promotion of development awareness (non-sector...,998.0


Split & explode the `ad_sector_codes` pipe‑delimited string into one row per code.

In [11]:
wb_proj = (
    wb_proj
    .assign(
        ad_sector_codes = wb_proj["ad_sector_codes"]
                          .str.split(r"\|")
    )
    .explode("ad_sector_codes")
    .reset_index(drop=True)
).astype({'ad_sector_codes': int})

Join on `sect_code` to pull in the `parent_code` and original `sect_code_name`.

In [12]:
wb_proj = wb_proj.merge(
    oecd_sect_df,
    left_on="ad_sector_codes",
    right_on="sect_code",
    how="left"
)

Roll up: if there *is* a `parent_code`, use that; otherwise keep the original.

In [13]:
wb_proj["ad_sector_top"] = np.where(
    wb_proj["parent_code"].isna(),
    wb_proj["ad_sector_codes"],
    wb_proj["parent_code"]
)

Deduplicate so each project‐location × sector is only one row.

In [14]:
wb_proj = (
    wb_proj
    .loc[:, wb_proj.columns.difference(
        ["ad_sector_codes", "sect_code", "parent_code", "code_name_e"]
    ) | ["ad_sector_top"]]
    .drop_duplicates()
)

/local/tmp.4856116/ipykernel_125576/1113066978.py:3: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  .loc[:, wb_proj.columns.difference(


Re‑join to get the human‐readable sector name for your rolled‑up code, then clean up columns.

In [15]:
wb_proj = (
    wb_proj
    .merge(
        oecd_sect_df[["sect_code", "sect_code_name"]],
        left_on="ad_sector_top",
        right_on="sect_code",
        how="left"
    )
    .rename(
        columns={
            "ad_sector_top": "ad_sector_codes",
            "sect_code_name": "ad_sector_names"
        }
    )
    .drop(columns=["sect_code"])
    .reset_index(drop=True)
)

Map each numeric sector code to its top‐level group (`SIS`, `EIS`, etc.) and give each group a name.

In [16]:
group_map = {
    #Social Infrastructure & Services
    110: "SIS", # Education
    120: "SIS", # Health
    130: "SIS", # Population Policies / Programmes and Reproductive Health
    140: "SIS", # Water Supply and Sanitation
    150: "SIS", # Government and Civil Society
    160: "SIS", # Other Social Infrastructure and Services
    # Economic Infrastructure & Services 
    210: "EIS", # Transport and Storage
    220: "EIS", # Communications
    230: "EIS", # Energy Generation and Supply
    240: "EIS", # Banking and Financial Services
    250: "EIS", # Business and Other Services
    # Production
    310: "PRO", # Agriculture, Forestry and Fishing
    320: "PRO", # Industry, Mining, Construction
    330: "PRO", # Trade and Tourism
    # Direct Aid
    520: "DIR", # Developmental Food Aid/Food Security Assistance
    700: "DIR", # Emergency Response
    600: "DIR", # Action Relating to Debt
    920: "DIR", # Support to Non-governmental Organizations (NGOs) and Government Organizations
    530: "DIR", # Non-food commodity assistance
    # Other
    410: "OTH", # General Environment Protection
    420: "OTH", # Women in Development
    430: "OTH", # Other Multisector
    998: "OTH" # Unallocated / Unspecified
}
name_map = {
    "SIS": "Social Infrastructure & Services",
    "EIS": "Economic Infrastructure & Services",
    "PRO": "Production",
    "DIR": "Direct Aid",
    "OTH": "Other"
}

# Ensure ad_sector_codes is numeric for mapping
wb_proj["ad_sector_codes"] = wb_proj["ad_sector_codes"].astype(int)

wb_proj["sector_group"] = wb_proj["ad_sector_codes"].map(group_map)
wb_proj["sector_group_name"] = wb_proj["sector_group"].map(name_map)

Update sector names to be consistent between WB and China

In [17]:
wb_proj["ad_sector_names"] = (
    wb_proj["ad_sector_names"]
    .replace({
        "Other Social infrastructure and services": "Other Social Infrastructure and Services",
        "General Environment Protection":        "General Environmental Protection",
        "Energy":                                "Energy Generation and Supply",
        "Agriculture, Forestry, Fishing":        "Agriculture, Forestry and Fishing"
    })
)

In [18]:
wb_proj

,ad_sector_names,ad_sector_codes,country,donors,donors_iso3,end_actual_isodate,end_actual_type,gazetteer_adm_code,gazetteer_adm_name,geographic_exactness,...,start_actual_isodate,start_actual_type,status,total_commitments,total_disbursements,transactions_end_year,transactions_start_year,sect_code_name_y,sector_group,sector_group_name
0,Energy generation and supply,230,Benin,World Bank,DAC,2016-06-30,end-actual,6295630|6255146|BJ|00,Earth|Africa|Benin|Avrankou,1,...,2009-06-23,start-actual,Implementation,7.244753e+07,1.024126e+06,2016,2009,Energy,EIS,Economic Infrastructure & Services
1,Energy generation and supply,230,Benin,World Bank,DAC,2016-06-30,end-actual,6295630|6255146|BJ|00,Earth|Africa|Benin|Badékparou,1,...,2009-06-23,start-actual,Implementation,7.244753e+07,1.024126e+06,2016,2009,Energy,EIS,Economic Infrastructure & Services
2,Energy generation and supply,230,Benin,World Bank,DAC,2016-06-30,end-actual,6295630|6255146|BJ|00,Earth|Africa|Benin|Bakpérou,1,...,2009-06-23,start-actual,Implementation,7.244753e+07,1.024126e+06,2016,2009,Energy,EIS,Economic Infrastructure & Services
3,Energy generation and supply,230,Benin,World Bank,DAC,2016-06-30,end-actual,6295630|6255146|BJ|00,Earth|Africa|Benin|Boroné,1,...,2009-06-23,start-actual,Implementation,7.244753e+07,1.024126e+06,2016,2009,Energy,EIS,Economic Infrastructure & Services
4,Energy generation and supply,230,Benin,World Bank,DAC,2016-06-30,end-actual,6295630|6255146|BJ|00,Earth|Africa|Benin|Démon,1,...,2009-06-23,start-actual,Implementation,7.244753e+07,1.024126e+06,2016,2009,Energy,EIS,Economic Infrastructure & Services
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20986,Energy generation and supply,230,Burkina Faso,World Bank,DAC,2019-09-30,end-actual,6295630|6255146|BF|09|2360075|2362344,Earth|Africa|Burkina Faso|High-Basins Region|P...,1,...,2013-07-30,start-actual,Implementation,5.164569e+07,NaN,2019,2013,Energy,EIS,Economic Infrastructure & Services
20987,Energy generation and supply,230,Burkina Faso,World Bank,DAC,2019-09-30,end-actual,6295630|6255146|BF|08|2360223|2360886,Earth|Africa|Burkina Faso|Est|Province du Gour...,1,...,2013-07-30,start-actual,Implementation,5.164569e+07,NaN,2019,2013,Energy,EIS,Economic Infrastructure & Services
20988,Energy generation and supply,230,Burkina Faso,World Bank,DAC,2019-09-30,end-actual,6295630|6255146|BF|05|2355914|2359608,Earth|Africa|Burkina Faso|Centre-Nord|Province...,1,...,2013-07-30,start-actual,Implementation,5.164569e+07,NaN,2019,2013,Energy,EIS,Economic Infrastructure & Services
20989,Energy generation and supply,230,Burkina Faso,World Bank,DAC,2019-09-30,end-actual,6295630|6255146|BF|10|2353568|2357043,Earth|Africa|Burkina Faso|Nord|Province du Yat...,1,...,2013-07-30,start-actual,Implementation,5.164569e+07,NaN,2019,2013,Energy,EIS,Economic Infrastructure & Services


## China Project Data (AidData 1.1.1)

Load raw China ODA

In [19]:
ch_proj = pd.read_csv(os.path.join(aid_data_dir, 'data/AiddataChinav1.1.1/GeoCoded_China_Data_Merged_Files/oda-like_flows.csv'))

Filter to precision ≤3, non‐umbrella, years 2002–2014, and non‐missing lat/long

In [20]:
ch_proj = ch_proj.loc[
    (ch_proj["precision_code"] <= 3) &
    (ch_proj["umbrella"] == False) &
    (ch_proj["transactions_start_year"] >= 2002) &
    (ch_proj["transactions_start_year"] <= 2014) &
    (ch_proj["latitude"].notna())
].copy()

This dataset doesn't have a "continent" column to filter by, so instead, we only keep entries where the recipient is in the DHS dataset

In [21]:
dhs_iso3s = df['iso3'].unique()
ch_proj = ch_proj[ch_proj["recipients_iso3"].isin(dhs_iso3s)].reset_index(drop=True)

Extract ISO2 from gazetteer_adm_code

In [22]:
ch_proj["site_iso2"] = ch_proj["gazetteer_adm_code"].str.extract(r"\|([A-Z]{2})\|", expand=False)

Remove the known outliers whose gazetteer places them in Costa Rica, Brazil, Rep. of Congo and Sudan

In [23]:
outliers = ["31253_3399415", "1020_3623076", "36333_2257266", "36320_2260535", "35629_11592387"]
ch_proj = ch_proj.loc[~ch_proj["project_location_id"].isin(outliers)].reset_index(drop=True)

Get ISO3 code and country name

In [24]:
ch_proj["site_iso3"] = ch_proj["site_iso2"].apply(lambda x: iso2_to_iso3s[x])
ch_proj['country'] = ch_proj['site_iso3'].apply(lambda x: iso3_to_dhs_country[x])

Map to get sector_group and sector_group_name

In [25]:
ch_proj["sector_group"] = ch_proj["ad_sector_codes"].map(group_map)
ch_proj["sector_group_name"] = ch_proj["sector_group"].map(name_map)

Harmonize ad_sector_names to match WB naming conventions

In [26]:
ch_proj["ad_sector_names"] = ch_proj["ad_sector_names"].replace({
    "Other Social infrastructure and services": "Other Social Infrastructure and Services",
    "General Environment Protection":         "General Environmental Protection",
    "Energy":                                 "Energy Generation and Supply",
    "Agriculture, Forestry, Fishing":         "Agriculture, Forestry and Fishing"
})

In [27]:
ch_proj

,project_id,project_location_id,precision_code,geoname_id,place_name,latitude,longitude,location_type_code,location_type_name,gazetteer_adm_code,...,location_details,contacts,source_triangulation,field_completeness,round_coded,site_iso2,site_iso3,country,sector_group,sector_group_name
0,31023,31023_11550981,1,11550981,Olkaria Geothermal Power Plant,0.89333,36.30833,PS,power station,6295630|6255146|KE|10|11550981,...,"Olkaria, Menengai and Silali","Davis Chirchir, Energy and Petroleum secretary...",8,8,AdditionalCoded,KE,KEN,Kenya,EIS,Economic Infrastructure & Services
1,31023,31023_11550982,1,11550982,Menengai Geothermal Power Station,0.19306,36.07000,PS,power station,6295630|6255146|KE|10|11550982,...,"Olkaria, Menengai and Silali","Davis Chirchir, Energy and Petroleum secretary...",8,8,AdditionalCoded,KE,KEN,Kenya,EIS,Economic Infrastructure & Services
2,23142,23142_158684,1,158684,Kabanga,-2.63861,30.46778,PPL,populated place,6295630|6255146|TZ|19|151710|11004435|158684,...,NaN,"Mr. Fu Jijun, Deputy Head and Political Counse...",12,9,ChinatoAfrica,TZ,TZA,Tanzania,EIS,Economic Infrastructure & Services
3,23142,23142_150387,1,150387,Rusumo,-2.38500,30.78583,PPL,populated place,6295630|6255146|TZ|19|151710|11004560|150387,...,NaN,"Mr. Fu Jijun, Deputy Head and Political Counse...",12,9,ChinatoAfrica,TZ,TZA,Tanzania,EIS,Economic Infrastructure & Services
4,23142,23142_161154,1,161154,Biharamulo,-2.63194,31.30889,PPL,populated place,6295630|6255146|TZ|19|161153|11004388|161154,...,NaN,"Mr. Fu Jijun, Deputy Head and Political Counse...",12,9,ChinatoAfrica,TZ,TZA,Tanzania,EIS,Economic Infrastructure & Services
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211,41576,41576_11611137,1,11611137,Kapps Farm,-22.54793,17.25826,FRMS,farms,6295630|6255146|NA|21|11611137,...,NaN,NaN,4,6,AdditionalCoded,NA,NAM,Namibia,EIS,Economic Infrastructure & Services
1212,41578,41578_11593266,1,11593266,Hardap Inland Aquaculture Center,-24.49409,17.86285,AQC,aquaculture facility,6295630|6255146|NA|30|11593266,...,NaN,"Xin Shunkang, Chinese Ambassador to Namibia ; ...",2,7,AdditionalCoded,NA,NAM,Namibia,SIS,Social Infrastructure & Services
1213,41601,41601_891993,1,891993,Dunolly,-18.13062,31.03899,FRM,farm,6295630|6255146|ZW|04|891993,...,NaN,"Han Bing, Embassy Official (China)",1,5,AdditionalCoded,ZW,ZWE,Zimbabwe,SIS,Social Infrastructure & Services
1214,16801,16801_8378550,3,8378550,Huye District,-2.58200,29.70200,ADM2H,historical second-order administrative division,6295630|6255146|RW|01,...,"Huye District, Southern Province, Rwanda","Shu Zhan, ; Habumuremyi , ; Daphrose Gahakwa, ...",8,9,EcologicallySensitive,RW,RWA,Rwanda,PRO,Production


## Merge the two

Find the intersection of column names and add new column "funder"

In [28]:
common_cols = list(
    set(wb_proj.columns)
    .intersection(ch_proj.columns)
) + ['funder']

Tag each dataset and select only those columns

In [29]:
wb_proj = wb_proj.assign(
    funder = "WB",
    project_id = wb_proj["project_id"].astype(str)
    ).loc[:, common_cols].copy()

ch_proj = ch_proj.assign(
    funder = "CH",
    project_id = ch_proj["project_id"].astype(str)
).loc[:, common_cols].copy()

Concatenate to one dataframe

In [30]:
oda_sect_group_df = pd.concat(
    [ch_proj, wb_proj],
    ignore_index=True,
    sort=False
)

In [33]:
from mapboxgl.utils import df_to_geojson, create_color_stops
from mapboxgl.viz import CircleViz

# Convert dataframe to GeoJSON
geojson = df_to_geojson(
    df=oda_sect_group_df,
    lat='latitude',
    lon='longitude',
    precision=6,
    properties=['sector_group_name']
)

# Map each unique sector group name to a specific color
sector_group_colors = [
    ['Social Infrastructure & Services', 'rgb(256,0,0)'],
    ['Economic Infrastructure & Services', 'rgb(0,256,0)'],
    ['Production', 'rgb(0,0,256)'],
    ['Direct Aid', 'rgb(256,256,0)'],
    ['Other', 'rgb(256,0,256)']
]

# Create the viz from the dataframe
viz = CircleViz(
    geojson,
    access_token=MAPBOX_TOKEN,
    color_property='sector_group_name',
    color_stops=sector_group_colors,
    color_function_type='match',
    radius=2.5,
    stroke_color='#FFFFFF',
    stroke_width=1,
    opacity=0.8,
    style='mapbox://styles/mapbox/streets-v11'
)
viz.show()

/opt/conda/lib/python3.8/site-packages/IPython/core/display.py:419: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


# Add ADM2 label to all projects

Load ADM2 polygons from the shapefile

In [34]:
import fiona
import shapely.geometry
from shapely.geometry import shape, Point
import pandas as pd
from rtree import index
import math

shapes = []
properties = []
with fiona.open(os.path.join(aid_data_dir, 'data/country_regions/gadm2_clean.shp')) as shp:
    for feature in tqdm(shp):
        geom = shape(feature['geometry'])
        shapes.append(geom)
        properties.append(feature['properties'])

100%|██████████| 47041/47041 [00:59<00:00, 794.30it/s] 


Build spatial index for polygons

In [35]:
idx = index.Index()
for i, geom in enumerate(shapes):
    idx.insert(i, geom.bounds)

Function to find the region for a point

In [36]:
def find_region(lat, lon):
    point = Point(lon, lat)

    # Try to find containing polygon
    candidates = list(idx.intersection((lon, lat, lon, lat)))
    for i in candidates:
        if shapes[i].contains(point):
            return properties[i]  # return ADM2 properties

    # If not found, find the nearest polygon
    min_dist = math.inf
    nearest_i = None
    for i in idx.nearest((lon, lat, lon, lat), 5):  # Try 5 nearest first
        dist = point.distance(shapes[i])
        if dist < min_dist:
            min_dist = dist
            nearest_i = i
    return properties[nearest_i] if nearest_i is not None else None

Get ADM2 region for each entry in `oda_sect_group_df`

In [37]:
oda_sect_group_df['ID_adm2'] = oda_sect_group_df.progress_apply(lambda row: find_region(row['latitude'], row['longitude']).get('ID_adm2'), axis=1)

100%|██████████| 22207/22207 [01:30<00:00, 244.08it/s] 


In [38]:
df['ID_adm2'] = df.progress_apply(lambda row: find_region(row['lat'], row['lon']).get('ID_adm2'), axis=1)

100%|██████████| 68619/68619 [04:58<00:00, 229.86it/s] 


Add start date to each project. Defaults to `start_actual_isodate`, but uses `transactions_start_year`if not available.

In [39]:
start_date = pd.to_datetime(oda_sect_group_df['start_actual_isodate'])
start_date[start_date.isna()] = pd.to_datetime(oda_sect_group_df[start_date.isna()]['transactions_start_year'].apply(lambda x: f'{x}-01-01'))
oda_sect_group_df['start_date'] = start_date

In [40]:
interventions_df = oda_sect_group_df[['project_location_id', 'start_date', 'ID_adm2']].drop_duplicates()
clusters_df = df[['cluster_id', 'year', 'month', 'ID_adm2']].copy()
clusters_df.loc[:, 'month'] = clusters_df['month'].clip(lower=1)
clusters_df.loc[:, 'survey_date'] = pd.to_datetime(clusters_df['year'].astype(str) + '-' + clusters_df['month'].astype(str) + '-01')
clusters_df = clusters_df.drop(columns=['year', 'month'])

intervention_dhs_link_df = interventions_df.merge(clusters_df, on='ID_adm2')
intervention_dhs_link_df.loc[:, 'years_since_intervention'] = (intervention_dhs_link_df['survey_date'] - intervention_dhs_link_df['start_date'])  / pd.Timedelta('365.2425 days')
intervention_dhs_link_df = intervention_dhs_link_df[intervention_dhs_link_df['years_since_intervention'] >= 0][['project_location_id', 'cluster_id', 'years_since_intervention']].reset_index(drop=True)
intervention_dhs_link_df

,project_location_id,cluster_id,years_since_intervention
0,31023_11550981,KE.Baringo.8B.1033,9.412924
1,31023_11550981,KE.Baringo.8B.1034,9.412924
2,31023_11550981,KE.Baringo.8B.1053,9.412924
3,31023_11550981,KE.Baringo.8B.1054,9.495062
4,31023_11550981,KE.Rift Valley.72.1155,1.664647
...,...,...,...
266575,P091077_328649,ET.Affar.81.35,11.589560
266576,P091077_328649,ET.Affar.81.36,11.589560
266577,P091077_328649,ET.Affar.81.37,11.589560
266578,P091077_328649,ET.Affar.81.38,11.589560


Save CSV linking project location to DHS clusters

In [42]:
os.makedirs(os.path.join(aid_data_dir, 'interim'), exist_ok=True)

intervention_dhs_link_df.to_csv(
    os.path.join(aid_data_dir, 'interim/intervention_dhs_link.csv'),
    index=False
)

Save compressed version of interverntions data

In [43]:
interventions_df = oda_sect_group_df[['project_id', 'project_location_id', 'country', 'latitude', 'longitude', 'start_date', 'funder', 'ad_sector_codes', 'sector_group_name']]
interventions_df = interventions_df.astype({'start_date': str})
interventions_df.to_csv(
    os.path.join(aid_data_dir, 'interim/interventions.csv'),
    index=False
)